# Moving circle



In [1]:
import numpy as np
import cv2

import panel as pn
import bokeh.io
import bokeh.plotting
import colorcet

# Image processing tools
import skimage
import skimage.io
import bebi103
import imageio

bokeh.io.output_notebook()
pn.extension()

C:\Users\Zijie\anaconda3\lib\site-packages\arviz\__init__.py:317: UserWarning: Trying to register the cmap 'cet_gray' which already exists.
  register_cmap("cet_" + name, cmap=cmap)
C:\Users\Zijie\anaconda3\lib\site-packages\arviz\__init__.py:317: UserWarning: Trying to register the cmap 'cet_gray_r' which already exists.
  register_cmap("cet_" + name, cmap=cmap)
C:\Users\Zijie\anaconda3\lib\site-packages\bebi103\viz.py:38: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

In [2]:
def draw_circle(stack_size,
                 xy0=(200,400),
                 xyf=(1080,400),
                 radius=100,
                 canvas_size=(800, 1280)
                ): 
    """
    Function to draw a dynamic circle
    for active matter experiments.
    
    Parameters
    -----------------------------------
    stack_size : int.
        number of frames in stack
    mov : list.
        coordinates to change per frame
    radius : int.
        radius of the circle
    center : tuple.
        coordinates of the center
    canvas_size : tuple.
        pixel area to fill
    
    Returns
    -----------------------------------
    plots : np.array
        list of arrays containing each 
        frame.
    """
    
    x0, y0 = xy0
    xf, yf = xyf
    
    #calculate steps
    x_step = (xf-x0)/stack_size
    y_step = (yf-y0)/stack_size

    
    #filled circle
    thickness = -1
    color = (255,255,255)
    center = np.array([x0,y0])
    
    plots = []
    for i in range(stack_size):
        img = np.zeros(canvas_size, dtype='uint8') 
        circle = np.flipud(cv2.circle(img, (int(center[0]), int(center[1])), radius, color, thickness))
        plots.append(circle)
        center = center + np.array([x_step,y_step])
    return np.asarray(plots)


In [3]:
def export_tiff(filename ,*patterns):
    """
    Function that concatenates dynamic
    patterns generated with draw_circle(),
    and exports the stack as a tiff file. 
    
    Parameters
    -----------------------------------
    filename : str.
        tiff file desired name
    *patterns : np.array
        patterns created with draw_circle
        function
    
    Returns
    -----------------------------------
    Pattern in working directory
    """
    
    stack = np.concatenate([*patterns])
     
    filename = filename + '.tiff'
    
    return imageio.mimwrite(filename, stack) 

In [6]:
# file name
filename = "pattern"

# patterns to concatenate
p1 = draw_circle(200, xy0=(0,0), xyf=(1080,400))
p2 = draw_circle(200, xy0=(1080,400), xyf=(300,200))

# export tiff
export_tiff(filename, p1, p2)

In [7]:
# visualize pattern (jupyter only)
ic = skimage.io.ImageCollection(f'{filename}.tiff', conserve_memory=True)
skimage.io.concatenate_images(ic)

def display_frame(frame=0):
    """
    Returns a movie in which each frame corresponds to 1 minute
    """
    # uncomment to display microns per pixel
    
    
    
    p = bebi103.image.imshow(
        ic[frame],
        frame_height=200,
        frame_width=320,
        cmap=colorcet.gray,)
#         interpixel_distance=0.5884,
#         length_units='µm/pixel')
    
    return p 

pn.interact(display_frame, frame=(0, len(ic)))

Column
    [0] Column
        [0] IntSlider(end=400, name='frame')
    [1] Row
        [0] Bokeh(Figure, name='interactive01561')